# Import Demand

In [1]:
map_file_name = 'data/disciplinas/dia_hora_instituto_ponto_linha_18h.json'
hora = 1080
sentido = 'ida'
acao = 'desembarque'
def intervalo_horario(horario):
    return '15:30' <= horario <= '17:30'
def intervalo_horario_fim(horario):
    return '17:30' <= horario <= '19:00'
def intervalo_horario_am_pm(horario):
    return '3:30:00 PM' <= horario <= '5:30:00 PM'

## Aulas MATRUSP

##### Read jsons

In [2]:
import os
import json

In [3]:
files = []
dir_path = 'data/disciplinas/json/'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

In [4]:
len(files)

5205

In [5]:
disciplinas = []
for file in files:
    with open(dir_path + file, 'r+') as f:
        content = json.loads(f.read())
        disciplinas.append(content)

In [6]:
disciplinas[1000]

{'unidade': 'Escola Politécnica',
 'departamento': 'Disciplinas Interdepartamentais \x96 Engenharia Civil e Ambiental (PCC - PEF - PHA \x96 PTR)',
 'campus': 'São Paulo',
 'codigo': '0313102',
 'nome': 'Introdução ao Projeto na Engenharia',
 'creditos_aula': 4,
 'creditos_trabalho': 1,
 'objetivos': 'De modo integrado e complementar à Disciplina Introdução à Engenharia Civil e Ambiental, conhecer e treinar procedimentos que envolvem métodos analíticos e sintéticos para atuar na engenharia, ferramentas de design e planejamento na engenharia, além de outros tópicos que possam ser necessários visando a inovação, sustentabilidade e empreendedorismo.',
 'programa_resumido': 'O contexto geral do desenvolvimento de projeto na engenharia; as fases de concepção (design) e de implementação de projetos de engenharia;  empreendedorismo e inovação; concepção na engenharia (Design Thinking); modelagem na engenharia; leitura e interpretação de representações.',
 'turmas': [{'codigo': '2022201',
   'i

##### Filtrar Institutos CUASO

In [7]:
fora_cuaso = ['Escola de Artes, Ciências e Humanidades', 'Escola de Enfermagem', 'Faculdade de Direito', 'Faculdade de Medicina',
              'Faculdade de Saúde Pública', 'Faculdade de Saúde Pública e Faculdade de Ciências Farmacêuticas', 'Escola de Enfermagem de Ribeirão Preto',
              'Museu de Zoologia']
def is_disciplina_cuaso(disciplina, fora_cuaso):
    if disciplina['campus'] != 'São Paulo':
        return False
    if disciplina['unidade'] in fora_cuaso:
        return False
    return True

In [8]:
disciplinas_cuaso = []
for disciplina in disciplinas:
    if is_disciplina_cuaso(disciplina, fora_cuaso):
        disciplinas_cuaso.append(disciplina)
len(disciplinas_cuaso)

2352

In [9]:
unidades_matrusp = {}
for disciplina in disciplinas_cuaso:
    unidades_matrusp[disciplina['unidade']] = 1
unidades_matrusp

{'Escola Politécnica': 1,
 'Instituto de Química': 1,
 'Escola de Comunicações e Artes': 1,
 'Instituto de Psicologia': 1,
 'Faculdade de Economia, Administração, Contabilidade e Atuária': 1,
 'Faculdade de Medicina Veterinária e Zootecnia': 1,
 'Instituto Oceanográfico': 1,
 'Instituto de Biociências': 1,
 'Faculdade de Filosofia, Letras e Ciências Humanas': 1,
 'Instituto de Matemática e Estatística': 1,
 'Instituto de Geociências': 1,
 'Faculdade de Educação': 1,
 'Escola de Educação Física e Esporte': 1,
 'Faculdade de Odontologia': 1,
 'Faculdade de Arquitetura e Urbanismo': 1,
 'Faculdade de Ciências Farmacêuticas': 1,
 'Instituto de Física': 1,
 'Instituto de Energia e Ambiente': 1,
 'Instituto de Ciências Biomédicas': 1,
 'Instituto de Relações Internacionais': 1,
 'Instituto de Estudos Brasileiros': 1,
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 1,
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 1,

##### Gerar Lista de Aulas

In [10]:
aulas = []
for i, disciplina in enumerate(disciplinas_cuaso):
    for turma in disciplina['turmas']:
        total_matriculados = 0
        for k, vaga in turma['vagas'].items():
            total_matriculados += vaga['matriculados']
            
        if turma['horario'] != None:
            for horarios in turma['horario']:
                aula = {}
                aula['indice_disc'] = i
                aula['codigo'] = disciplina['codigo']
                aula['codigo_turma'] = turma['codigo']
                aula['unidade'] = disciplina['unidade']
                aula['dia'] = horarios['dia']
                aula['horario_inicio'] = horarios['inicio']
                aula['horario_fim'] = horarios['fim']
                aula['total_matriculados'] = total_matriculados
                aulas.append(aula)

##### Filtrar Aulas por Intervalo Horário

In [11]:
aulas_intervalo_horario = []
for aula in aulas:
    if intervalo_horario(aula['horario_inicio']) and intervalo_horario_fim(aula['horario_fim']):
        aulas_intervalo_horario.append(aula)

# Pesquisa

##### Ler Pesquisa

In [12]:
import pandas as pd
import datetime as dt

In [13]:
pesquisa_perguntas = pd.read_csv('./data/disciplinas/forms/resposta_pesq.csv', delimiter=';')
pesquisa_perguntas['timestamp'] = list(map(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), pesquisa_perguntas['Carimbo de data/hora']))
pesquisa_perguntas = pesquisa_perguntas[pesquisa_perguntas['timestamp'] >= dt.datetime(2022, 10, 7, 0, 0, 0)]
len(pesquisa_perguntas)

4395

In [14]:
pesquisa = pd.DataFrame()
pesquisa['vinculo'] = pesquisa_perguntas['Qual o seu vínculo com a USP?']
pesquisa['instituto'] = pesquisa_perguntas['A qual instituto você está associado?']


# ida
pesquisa['utiliza_circular_ida'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para entrar na USP?'] >= 2
pesquisa['linhas_ida'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?']
pesquisa['ponto_embarque_ida'] = pesquisa_perguntas['Por qual ponto você normalmente embarca?']
pesquisa['ponto_desembarque_ida'] = pesquisa_perguntas['Em qual ponto de ônibus você normalmente desembarca?']
pesquisa['horario_ida'] = pesquisa_perguntas['Que horas você normalmente chega na USP?']


# volta
pesquisa['utiliza_circular_volta'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para sair da USP?'] >= 2
pesquisa['linhas_volta'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?.1']
pesquisa['ponto_embarque_volta'] = pesquisa_perguntas['Em qual o ponto de ônibus você embarca para sair da USP?']
pesquisa['ponto_desembarque_volta'] = pesquisa_perguntas['Por qual ponto você normalmente desembarca?']
pesquisa['horario_volta'] = pesquisa_perguntas['Que horas você normalmente chega no ponto de ônibus para sair da USP?']

pesquisa.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


##### Filtrar estudantes

In [15]:
pesquisa[['vinculo']].value_counts()

vinculo                                                        
Aluno de graduação                                                 2880
Aluno de pós-graduação                                              752
Funcionário                                                         644
Docente                                                              36
docente                                                              22
Professor                                                            21
professor                                                             8
Ex-aluno                                                              5
Professor                                                             5
Docente                                                               4
Professor titular                                                     1
aprimoramento                                                         1
Pós-doutorando                                                        1


In [16]:
pesquisa[['vinculo', 'utiliza_circular_' + sentido]].value_counts()

vinculo                                                          utiliza_circular_ida
Aluno de graduação                                               True                    2532
Aluno de pós-graduação                                           True                     575
Funcionário                                                      True                     369
Aluno de graduação                                               False                    348
Funcionário                                                      False                    275
Aluno de pós-graduação                                           False                    177
Docente                                                          True                      28
docente                                                          False                     17
Professor                                                        True                      11
                                                                 Fal

In [17]:
def grad_ou_pos(x):
    return x == 'Aluno de graduação' or x == 'Aluno de pós-graduação'
pesquisa_alunos = pesquisa[list(map(grad_ou_pos, pesquisa['vinculo']))]
pesquisa_alunos[['vinculo']].value_counts()

vinculo               
Aluno de graduação        2880
Aluno de pós-graduação     752
dtype: int64

##### Filtrar respostas pesquisa pelo Intervalo Horário

In [18]:
pesquisa_alunos.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [19]:
pesquisa_data = pesquisa_alunos[pesquisa_alunos['horario_' + sentido].notna()]
pesquisa_alunos_intervalo = pesquisa_data[list(map(intervalo_horario_am_pm, pesquisa_data['horario_' + sentido]))]
pesquisa_alunos_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
158,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8022, 8032",Estação Butantã,FEA,4:30:00 PM,True,8022,FAU II,Portão da Vila Indiana,10:30:00 PM
198,Aluno de graduação,Instituto de Física (IF),True,"8012, 8022",P3,Prefeitura/Física,5:25:00 PM,True,8012,Prefeitura/Física,P3,10:40:00 PM
199,Aluno de graduação,"Faculdade de Economia, Administração e Contabi...",True,"8012, 8032",Estação Butantã,FEA,4:30:00 PM,True,8032,FEA,Estação Butantã,10:50:00 PM
239,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,4:15:00 PM,True,8032,Biblioteca Brasiliana,Estação Butantã,10:15:00 PM
289,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,Estação Butantã,História e Geografia,3:55:00 PM,True,8032,História e Geografia,Estação Butantã,10:20:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
4384,Aluno de graduação,Escola de Comunicações e Artes (ECA),True,"8012, 8032",Estação Butantã,Reitoria/Bancos,5:30:00 PM,True,8012,ECA,Estação Butantã,10:00:00 PM
4432,Aluno de graduação,Instituto de Física (IF),True,8012,Estação Butantã,Física,4:20:00 PM,True,"8022, 8032",Física,Estação Butantã,11:10:00 PM
4435,Aluno de pós-graduação,Instituto de Ciências Biomédicas (ICB),True,8022,Estação Butantã,Acesso Vila Indiana,4:40:00 PM,True,"8012, 8022",Acesso Vila Indiana,Estação Butantã,6:00:00 PM
4477,Aluno de graduação,"Faculdade de Economia, Administração e Contabi...",True,"8012, 8032",PTREM,FEA,5:00:00 PM,True,"8012, 8032",ECA,PTREM,10:50:00 PM


##### Normaliza instituto com unidade do matrusp

In [20]:
def retira_sigla(x):
    index = x.find('(')
    
    if index < 0:
        return x
    
    return x[:index - 1]

def filtra_instituto_pesq(x):
    x = retira_sigla(x)
    
    if x in ['ECA e MAC USP', 'Escola de Artes, Ciências e Humanidades']: 
        return 'Escola de Comunicações e Artes'
    elif x == 'FFLCH':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x in ['IPEN', 'Instituto de Pesquisas Energéticas e Necleares', 'IPEN ', 'ipen', 'Instituto de pesquisas energéticas e nucleares', 'Instituto de Pesquisa Energéticas e Nucleares', 'Instituto de Pesquisas Energéticas e Nucleares - IPEN', 'Instituto de pesquisas energéticas e nucleares ', 'Instituto de pesquisas energéticas e nucleares -IPEN', 'Instituo de Pesquisas Energéticas e Nucleares']:
        return 'Instituto de Pesquisas Energéticas e Nucleares'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    elif x in ['PRG', 'Prg ', 'Pró-Reitoria de Graduação', 'Pró-Reitoria de graduação', 'Pró-reitoria de graduacao', 'Curso de Ciências Moleculares']:
        return None
    elif x == 'Faculadae de Arquitetura e Urbanismo':
        return 'Faculdade de Arquitetura e Urbanismo'
    elif x == 'Instituo de Astronomia, Geofísica e Ciências Atmosféricas':
        return 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas'
    elif x == 'Faculdade de Economia, Administração e Contabilidade':
        return 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    elif x in ['Escola de Aplicação da Faculdade de Educação (EA-FEUSP)']:
        return 'Escola de Educação Física e Esporte'
    elif x == 'Instituto de Relações internacionais':
        return 'Instituto de Relações Internacionais'
    elif x in ['Museu de arqueologia e etnologia ']:
        return 'Museu de Arqueologia e Etnologia'
    elif x in ['Instituto de Energia e Ambientes']:
        return 'Instituto de Energia e Ambiente'
    else:
        return x
    
pesquisa_alunos_intervalo2 = pesquisa_alunos_intervalo.copy()
pesquisa_alunos_intervalo2['instituto'] = list(map(filtra_instituto_pesq, pesquisa_alunos_intervalo['instituto']))

In [21]:
# Filtra os Nones

pesquisa_alunos_intervalo = pesquisa_alunos_intervalo2
pesquisa_alunos_intervalo = pesquisa_alunos_intervalo[pesquisa_alunos_intervalo['instituto'].notna()]
pesquisa_alunos_intervalo['instituto'].unique()

array(['Faculdade de Arquitetura e Urbanismo', 'Instituto de Física',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Faculdade de Ciências Farmacêuticas',
       'Instituto de Matemática e Estatística',
       'Escola de Comunicações e Artes',
       'Instituto de Pesquisas Energéticas e Nucleares',
       'Faculdade de Odontologia', 'Escola Politécnica',
       'Instituto de Relações Internacionais', 'Faculdade de Educação',
       'Instituto de Biociências', 'Instituto de Química',
       'Instituto de Geociências', 'Faculdade de Medicina',
       'Instituto de Ciências Biomédicas', 'Faculdade de Saúde Pública'],
      dtype=object)

##### Mapeia Institutos Fora CUASO

In [22]:
def mapeia_cursos_de_fora(x):
    if x in ['Faculdade de Medicina', 'Escola de Enfermagem', 'Faculdade de Saúde Pública']:
        return 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    elif x == 'Faculdade de Direito':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    else:
        return x
    
pesquisa_alunos_intervalo2 = pesquisa_alunos_intervalo.copy()
pesquisa_alunos_intervalo2['instituto'] = list(map(mapeia_cursos_de_fora, pesquisa_alunos_intervalo['instituto']))
pesquisa_alunos_intervalo = pesquisa_alunos_intervalo2

In [23]:
pesquisa_alunos_intervalo['instituto'].unique()

array(['Faculdade de Arquitetura e Urbanismo', 'Instituto de Física',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Faculdade de Ciências Farmacêuticas',
       'Instituto de Matemática e Estatística',
       'Escola de Comunicações e Artes',
       'Instituto de Pesquisas Energéticas e Nucleares',
       'Faculdade de Odontologia', 'Escola Politécnica',
       'Instituto de Relações Internacionais', 'Faculdade de Educação',
       'Instituto de Biociências', 'Instituto de Química',
       'Instituto de Geociências',
       'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
       'Instituto de Ciências Biomédicas'], dtype=object)

##### Filtrar Butantã

In [24]:
pesquisa_alunos_intervalo_butanta = pesquisa_alunos_intervalo[pesquisa_alunos_intervalo['utiliza_circular_' + sentido]]
if sentido == 'ida':
    def is_butanta(x): 
        return x == 'Estação Butantã' or x == 'PTREM'

    pesquisa_alunos_intervalo_butanta = pesquisa_alunos_intervalo_butanta[list(map(is_butanta, pesquisa_alunos_intervalo_butanta['ponto_embarque_ida']))]
pesquisa_alunos_intervalo_butanta

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
158,Aluno de graduação,Faculdade de Arquitetura e Urbanismo,True,"8012, 8022, 8032",Estação Butantã,FEA,4:30:00 PM,True,8022,FAU II,Portão da Vila Indiana,10:30:00 PM
199,Aluno de graduação,"Faculdade de Economia, Administração, Contabil...",True,"8012, 8032",Estação Butantã,FEA,4:30:00 PM,True,8032,FEA,Estação Butantã,10:50:00 PM
239,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,"8012, 8032",Estação Butantã,Biblioteca Brasiliana,4:15:00 PM,True,8032,Biblioteca Brasiliana,Estação Butantã,10:15:00 PM
289,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,8012,Estação Butantã,História e Geografia,3:55:00 PM,True,8032,História e Geografia,Estação Butantã,10:20:00 PM
315,Aluno de graduação,Faculdade de Ciências Farmacêuticas,True,"8012, 8022",Estação Butantã,Biomédicas,5:20:00 PM,True,8022,Biomédicas,Estação Butantã,10:00:00 PM
...,...,...,...,...,...,...,...,...,...,...,...,...
4384,Aluno de graduação,Escola de Comunicações e Artes,True,"8012, 8032",Estação Butantã,Reitoria/Bancos,5:30:00 PM,True,8012,ECA,Estação Butantã,10:00:00 PM
4432,Aluno de graduação,Instituto de Física,True,8012,Estação Butantã,Física,4:20:00 PM,True,"8022, 8032",Física,Estação Butantã,11:10:00 PM
4435,Aluno de pós-graduação,Instituto de Ciências Biomédicas,True,8022,Estação Butantã,Acesso Vila Indiana,4:40:00 PM,True,"8012, 8022",Acesso Vila Indiana,Estação Butantã,6:00:00 PM
4477,Aluno de graduação,"Faculdade de Economia, Administração, Contabil...",True,"8012, 8032",PTREM,FEA,5:00:00 PM,True,"8012, 8032",ECA,PTREM,10:50:00 PM


##### Mapeia Pontos Pesquisa para ficar igual ao da API

In [25]:
import requests
pontos = requests.get('https://uspdigital.usp.br/mobile/servicos/mapa/locais?campus=cidade-universitaria&categorias=transportes')
pontos = json.loads(pontos.text)
pontos[10]

{'categoria': 'transportes',
 'campus': 'cidade-universitaria',
 'id': '1811',
 'titulo': 'Ponto de ônibus ECA',
 'latitude': '-23.557806338669685',
 'longitude': '-46.72689100000002',
 'endereco': 'Av. Prof. Lúcio Martins Rodrigues',
 'imagem': 'http://www.usp.br/mapas/wp-content/uploads/10042014giroculturalfotomarcossantos008.jpg',
 'url': '',
 'descricao': 'Atende as linhas  701U-10, 702U-10, 7725-10, 8012-10'}

In [26]:
nomes_pontos_req = []
for ponto in pontos:
    if ponto['campus'] == 'cidade-universitaria':
        nomes_pontos_req.append(ponto['titulo'])
 
nomes_pontos_pesq = []
for ponto in pesquisa_alunos_intervalo['ponto_' + acao + '_' + sentido].unique():
    nomes_pontos_pesq.append(ponto)

print(nomes_pontos_req)
print(nomes_pontos_pesq)

['Ponto de ônibus metrô Butantã', 'Ponto de ônibus Afrânio Peixoto', 'Ponto de ônibus Poli Metalúrgica', 'Terminal de ônibus USP', 'Ponto de ônibus Poli Mecânica', 'Ponto de ônibus portaria II', 'Ponto de ônibus Hidráulica', 'Ponto de Psicologia I', 'Ponto de ônibus Barracões', 'Ponto de ônibus ECA', 'Ponto de ônibus Praça do Relógio', 'Ponto de ônibus - Praça do Relógio', 'Ponto de ônibus Psicologia II', 'Ponto de ônibus - Acesso CPTM II', 'Ponto de ônibus - Acesso CPTM I', 'Ponto de ônibus Escola de Educação Física II', 'Ponto de ônibus Escola de Educação Física I', 'Ponto de ônibus Academia de Polícia', 'Ponto de ônibus Paços das Artes', 'Ponto de ônibus Educação', 'Ponto de ônibus CRUSP', 'Ponto de ônibus Cultura Japonesa', 'Ponto de ônibus Biblioteca Brasiliana', 'Ponto de ônibus Letras', 'Ponto de ônibus Geociência', 'Ponto de ônibus Reitoria/Bancos', 'Ponto de ônibus FEA', 'Ponto de ônibus FAU II', 'Ponto de ônibus Poli Biênio', 'Ponto de ônibus Poli Eletrotécnica', 'Ponto de ôn

In [27]:
def iguais_com_prefixo(x):
    aux_nomes_pontos_pesq = ['Ponto de ônibus ' + x for x in nomes_pontos_pesq]
    if 'Ponto de ônibus ' + x in [item for item in aux_nomes_pontos_pesq if item in list(set(nomes_pontos_req) & set(aux_nomes_pontos_pesq))]:
        return 'Ponto de ônibus ' + x
    else:
        return x
 
def pesquisa_to_pontos_req(x):
    if x in ["Geografia/História", "História e Geografia", "Químicas"]:
        return "Ponto e ônibus História e Geografia"
    elif x  == "Poli Metalurgia":
        return "Ponto de ônibus Poli Metalúrgica"
    elif x == "P3":
        return "Ponto de ônibus Portaria III"
    elif x == "Farmácia e Química":
        return "Ponto de ônibus Biblioteca Farmácia e Química"
    elif x in ["CRUSP (Praça da Reitoria)", "R. Prof. Mello Moraes, 1473 (localizado entre a CRUSP e a Raia, sentindo Metrô Butantã)", "R. Prof. Mello Moraes, 900 (localizado entre a CRUSP e a Raia, sentindo P3)"]:
        return "Ponto de ônibus CRUSP"
    elif x == "Biomédicas":
        return "Ponto de ônibus Biomédicas III"
    elif x in ["Prefeitura/Física", "Ponto de ônibus Ponto Clube dos funcionários"]:
        return "Ponto Clube dos funcionários"
    elif x == "Educação Física":
        return "Ponto de ônibus Escola de Educação Física I"
    elif x in ["Estação Butantã", "Terminal Cidade Universitária"]:
        return "Ponto de ônibus metrô Butantã"
    elif x == "Poli-Eletrotécnia":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x == "Geociências":
        return "Ponto de ônibus Geociências I"
    elif x in ["icb ", "Biociências", "ICB 1 (Av. Prof. Lineu Prestes, 1524)"]:
        return "Ponto de ônibus  Parada Matemática"
    elif x in ["FE", "Avenida da Universidade", "Faculdade de Educação ", "Educação / CEPEUSP", " FE", "Prédio da FE", "Educação ", "Educação", "Faculdade de Educação"]:
        return "Ponto de ônibus Educação"
    elif x in ["Rua do matão frente da FAU", "Instituto Oceanografico", "Oceanográfico, em frent ao da FAU II", "Rua do Matão, 0 (em frente a FAU)", "Estacionamento FAU", "R. do Matão, frente à FAU", "R. do Matão 878", "R. do Matão, 0", "Oceanográfico", "Oceanografia", "Rua do Matao ", "Matão, Oceanografico", "Rua do Matão, 0 - FAU", "IO USP", "IO", "Rua do matao, 0", "Rua do Matão 0 (FAU)", "Rua do Matão (na frente da FAU)", "Intuito Oceanografico", "R. do Matão", "Oceanografia ", "Oceanógrafico/Matão", "Rua do Matão, 0", "R. Do matão ",  "Rua do Matão", "Ponto em frente à FAU na Rua do Matão"]:
        return "Ponto de ônibus FAU I"
    elif x in ["depois da biblioteca da química ", "Rua do Lago", "Lineu Prestes entre a biblioteca da quimica e o icb", "Ponto do Lago", "Av. Prof. Lineu Prestes, 0"]:
        return "Ponto de ônibus rua do Lago"
    elif x == "Psicologia I":
        return "Ponto de ônibus Psicologia II"
    elif x in ["Cepam",  "Biomédicas, Biologia e Quimica"]:
        return "Ponto de ônibus CEPAM"
    elif x in ["Ipen", "FO ou Ipen", "Marinha/ Odontologia"]:
        return "Ponto de ônibus IPEN"
    elif x in ["Acesso Vila Indiana", "Portão da Vila Indiana"]:
        return "Ponto de ônibus acesso Vl. Indiana"
    elif x == "PTREM":
        return "Ponto de ônibus - Acesso CPTM I"
    elif x == "IPT":
        return "Ponto de ônibus Cocesp I"
    elif x in ["Centro De Difusão Internacional", "Poli Hidráulica"]:
        return "Ponto de ônibus Hidráulica"
    elif x == "Avenida Professor Luciano Gualberto (depois da Parada I.P.T, no 8022)":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x in ["P1", "Instituto Butantan ", "Academia de polícia"]:
        return "Ponto de ônibus Academia de Polícia"
    elif x == "P2":
        return "Ponto de ônibus portaria II"
    elif x in ["FAU ", "FEA ou FAU II", "As vezes desembarco na FAU II ou na FEA"]:
        return "Ponto de ônibus FAU II"
    elif x == "Portaria São Remo":
        return "Ponto de ônibus acesso Rio Pequeno"
    elif x == "If e Ipen":
        return "Ponto de ônibus Física"
    elif x == "Casa de Cultura Japonesa, ETEC CEPAM":
        return "Ponto de ônibus Cultura Japonesa"
    elif x == "IEB":
        return "Ponto de ônibus Biblioteca Brasiliana"
    elif x in ["IAG (ou, às vezes, Física)", "IAG ou Física", "IAG (sentido Metrô Butantã)"]:
        return "Ponto de ônibus IAG"
    elif x == "Prof Almeida Prado":
        return "Terminal de ônibus USP"
    elif x == "HU":
        return "Ponto de ônibus da COPESP"
    elif x == "pço das artes":
        return "Ponto de ônibus Paços das Artes"
    elif x == "Depende do dia":
        return None
    else:
        return x

In [28]:
pesquisa_alunos_intervalo_butanta['ponto_' + acao + '_' + sentido] = list(map(lambda x: pesquisa_to_pontos_req(iguais_com_prefixo(x)), pesquisa_alunos_intervalo_butanta['ponto_' + acao + '_' + sentido]))
pesquisa_alunos_intervalo_butanta = pesquisa_alunos_intervalo_butanta[pesquisa_alunos_intervalo_butanta['ponto_' + acao + '_' + sentido].notna()]
pesquisa_alunos_intervalo_butanta['ponto_' + acao + '_' + sentido].unique()

for ponto in pesquisa_alunos_intervalo_butanta['ponto_' + acao + '_' + sentido].unique():
    if ponto not in nomes_pontos_req:
        print(ponto)

Rua do lago
IQ


##### Calcular Porcentagem Carro por Instituto

In [29]:
porc_circular_por_instituto = pesquisa_alunos_intervalo_butanta[['instituto','utiliza_circular_' + sentido]].groupby('instituto').agg('mean').reset_index()
porc_circular_por_instituto

,instituto,utiliza_circular_ida
0,Escola Politécnica,1.0
1,Escola de Comunicações e Artes,1.0
2,Faculdade de Arquitetura e Urbanismo,1.0
3,Faculdade de Ciências Farmacêuticas,1.0
4,"Faculdade de Economia, Administração, Contabil...",1.0
5,Faculdade de Educação,1.0
6,"Faculdade de Filosofia, Letras e Ciências Humanas",1.0
7,"Faculdade de Medicina, Instituto de Ciências B...",1.0
8,Faculdade de Odontologia,1.0
9,Instituto de Biociências,1.0


In [30]:
def porc_utiliza_circular(instituto):
    return porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_' + sentido].iloc[0]
porc_utiliza_circular('Instituto de Física')

1.0

##### Calcula porcentagem para Ponto e Linha por Instituto (filtrando quem usa carro)

In [31]:
import random
def usa_linha(str_linhas, linha_target):
    linhas = str_linhas.split(', ')
    linha_selecionada = linhas[random.randint(0, len(linhas) - 1)]
    return (linha_selecionada == linha_target)

def calcula_quantidade_pessoas_ponto(instituto, ponto, linha):
    df_instituto = pesquisa_alunos_intervalo_butanta[pesquisa_alunos_intervalo_butanta['instituto'] == instituto]

    df_linha = df_instituto[list(map(lambda x: usa_linha(x, linha), df_instituto['linhas_' + sentido]))]    
    if len(df_linha['linhas_' + sentido]) == 0:
        return 0

    df_ponto = df_linha[df_linha['ponto_' + acao + '_' + sentido] == ponto]
    if len(df_ponto['ponto_' + acao + '_' + sentido]) == 0:
        return 0

    return df_ponto['ponto_' + acao + '_' + sentido].value_counts().iloc[0]

In [32]:
pontos_existentes = pesquisa_alunos_intervalo_butanta['ponto_' + acao + '_' + sentido].unique()

institutos_ponto_rel = {}
for instituto in pesquisa_alunos_intervalo_butanta['instituto'].unique():
    institutos_ponto_rel[instituto] = {}
    soma = len(pesquisa_alunos_intervalo_butanta[pesquisa_alunos_intervalo_butanta['instituto'] == instituto]['instituto'])
    for ponto in pontos_existentes:
        for linha in ['8012', '8022', '8032']:
            pessoas = calcula_quantidade_pessoas_ponto(instituto, ponto, linha) * porc_utiliza_circular(instituto)
            if pessoas > 0:
                if ponto not in institutos_ponto_rel[instituto]:
                    institutos_ponto_rel[instituto][ponto] = {}
                institutos_ponto_rel[instituto][ponto][linha] = pessoas / soma
institutos_ponto_rel

{'Faculdade de Arquitetura e Urbanismo': {'Ponto de ônibus FEA': {'8022': 0.3333333333333333}},
 'Faculdade de Economia, Administração, Contabilidade e Atuária': {'Ponto de ônibus FEA': {'8012': 0.3333333333333333,
   '8032': 0.6},
  'Ponto Clube dos funcionários': {'8012': 0.06666666666666667,
   '8032': 0.06666666666666667}},
 'Faculdade de Filosofia, Letras e Ciências Humanas': {'Ponto de ônibus Biblioteca Brasiliana': {'8012': 0.2558139534883721,
   '8022': 0.09302325581395349,
   '8032': 0.20930232558139536},
  'Ponto e ônibus História e Geografia': {'8012': 0.046511627906976744,
   '8032': 0.06976744186046512},
  'Ponto de ônibus CRUSP': {'8012': 0.09302325581395349,
   '8022': 0.046511627906976744,
   '8032': 0.023255813953488372},
  'Ponto de ônibus metrô Butantã': {'8032': 0.046511627906976744},
  'Ponto de ônibus Educação': {'8032': 0.023255813953488372},
  'Ponto de ônibus Letras': {'8012': 0.046511627906976744,
   '8022': 0.09302325581395349,
   '8032': 0.046511627906976744

# Pesquisa e Matrusp

##### Verificar unidades Matrusp e Pesquisa

In [33]:
# Alterar a unidade de aulas na qual a unidade não respondeu a pesquisa

for aula in aulas_intervalo_horario:
    if aula['unidade'] == 'Instituto de Ciências Biomédicas':
        aula['unidade'] = 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    if aula['unidade'] == 'Instituto de Estudos Brasileiros':
        aula['unidade'] = 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    if aula['unidade'] == 'Instituto de Psicologia':
        aula['unidade'] = 'Escola de Comunicações e Artes'

In [34]:
institutos_matrusp = {}
for aula in aulas_intervalo_horario:
    institutos_matrusp[aula['unidade']] = 1
institutos_matrusp = list(institutos_matrusp.keys())
institutos_matrusp.sort()
institutos_matrusp

['Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Escola de Educação Física e Esporte',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina Veterinária e Zootecnia',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Energia e Ambiente',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Química']

In [35]:
institutos_pesquisa = list(institutos_ponto_rel.keys())
institutos_pesquisa.sort()
institutos_pesquisa

['Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Instituto de Biociências',
 'Instituto de Ciências Biomédicas',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Química',
 'Instituto de Relações Internacionais']

In [36]:
for instituto_matrusp in institutos_matrusp:
    if instituto_matrusp not in institutos_pesquisa:
        print(instituto_matrusp + ' from matrusp is not in institutos pesquisa')

Escola de Educação Física e Esporte from matrusp is not in institutos pesquisa
Faculdade de Medicina Veterinária e Zootecnia from matrusp is not in institutos pesquisa
Instituto de Astronomia, Geofísica e Ciências Atmosféricas from matrusp is not in institutos pesquisa
Instituto de Energia e Ambiente from matrusp is not in institutos pesquisa


In [37]:
institutos_ponto_rel['Escola Politécnica']

{'Ponto de ônibus Poli Civil': {'8012': 0.5}}

##### Calcula Map: dia->hora->ponto->linha->pessoas

In [38]:
dia_hora_ponto_linha = {}

def sum_valor_dia_hora_ponto_linha(dia, hora, ponto, linha, valor):
    if dia not in dia_hora_ponto_linha:
        dia_hora_ponto_linha[dia] = {}
    if hora not in dia_hora_ponto_linha[dia]:
        dia_hora_ponto_linha[dia][hora] = {}
    if ponto not in dia_hora_ponto_linha[dia][hora]:
        dia_hora_ponto_linha[dia][hora][ponto] = {}
    if linha not in dia_hora_ponto_linha[dia][hora][ponto]:
        dia_hora_ponto_linha[dia][hora][ponto][linha] = 0
    dia_hora_ponto_linha[dia][hora][ponto][linha] += valor

for aula in aulas_intervalo_horario:
    if aula['unidade'] in institutos_ponto_rel:
        porcentagem_pontos_por_instituto = institutos_ponto_rel[aula['unidade']]
        for ponto in porcentagem_pontos_por_instituto:
            linhas = porcentagem_pontos_por_instituto[ponto]
            for linha in linhas:
                porc = linhas[linha]
                valor = aula['total_matriculados'] * porc
                sum_valor_dia_hora_ponto_linha(aula['dia'], hora, ponto, linha, valor)

# arredondando
for dia in dia_hora_ponto_linha:
    for hora in dia_hora_ponto_linha[dia]:
        for ponto in dia_hora_ponto_linha[dia][hora]:
            for linha in dia_hora_ponto_linha[dia][hora][ponto]:
                dia_hora_ponto_linha[dia][hora][ponto][linha] = round(dia_hora_ponto_linha[dia][hora][ponto][linha])

dia_hora_ponto_linha

{'sex': {1080: {'Ponto de ônibus Biblioteca Brasiliana': {'8012': 7,
    '8032': 7,
    '8022': 0},
   'Ponto de ônibus Biblioteca Farmácia e Química': {'8012': 6, '8022': 39},
   'IQ': {'8022': 6},
   'Ponto de ônibus FEA': {'8012': 28, '8032': 19},
   'Ponto de ônibus FAU II': {'8032': 9},
   'Ponto de ônibus Poli Eletrotécnica': {'8022': 19},
   'Ponto de ônibus Física': {'8012': 16},
   'Ponto Clube dos funcionários': {'8012': 11},
   'Ponto de ônibus Poli Civil': {'8012': 90},
   'Ponto de ônibus CRUSP': {'8022': 20, '8012': 0, '8032': 0},
   'Ponto de ônibus  Parada Matemática': {'8022': 20},
   'Rua do lago': {'8012': 20, '8022': 20},
   'Ponto e ônibus História e Geografia': {'8012': 0, '8032': 0},
   'Ponto de ônibus metrô Butantã': {'8032': 0},
   'Ponto de ônibus Educação': {'8032': 0},
   'Ponto de ônibus Letras': {'8012': 0, '8022': 0, '8032': 0},
   'Ponto de ônibus Psicologia II': {'8012': 0}}},
 'ter': {1080: {'Ponto de ônibus CRUSP': {'8022': 44, '8012': 3, '8032': 1},

##### Salva Map em Arquivo

In [39]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def encode(s):
    return s.encode('latin1').decode('unicode_escape')

with open(map_file_name, 'w+') as f:
    str_dia_hora_ponto_linha = encode(json.dumps(dia_hora_ponto_linha, indent=4, cls=NpEncoder))
    f.write(str_dia_hora_ponto_linha)
    print(str_dia_hora_ponto_linha)

{
    "sex": {
        "1080": {
            "Ponto de ônibus Biblioteca Brasiliana": {
                "8012": 7,
                "8032": 7,
                "8022": 0
            },
            "Ponto de ônibus Biblioteca Farmácia e Química": {
                "8012": 6,
                "8022": 39
            },
            "IQ": {
                "8022": 6
            },
            "Ponto de ônibus FEA": {
                "8012": 28,
                "8032": 19
            },
            "Ponto de ônibus FAU II": {
                "8032": 9
            },
            "Ponto de ônibus Poli Eletrotécnica": {
                "8022": 19
            },
            "Ponto de ônibus Física": {
                "8012": 16
            },
            "Ponto Clube dos funcionários": {
                "8012": 11
            },
            "Ponto de ônibus Poli Civil": {
                "8012": 90
            },
            "Ponto de ônibus CRUSP": {
                "8022": 20,
                "80

## Add employees and professors
TODO: considerar funcionários e professores estrapolando o horário de entrada da pesquisa

## Outros

##### Número Pessoas por Dia e Instituto

In [40]:
agrupamento_dia_instituto = {}
agrupamento_dia_instituto['seg'] = {}
agrupamento_dia_instituto['ter'] = {}
agrupamento_dia_instituto['qua'] = {}
agrupamento_dia_instituto['qui'] = {}
agrupamento_dia_instituto['sex'] = {}
agrupamento_dia_instituto['sab'] = {}
agrupamento_dia_instituto['dom'] = {}
for aula in aulas_intervalo_horario:
    if aula['unidade'] not in agrupamento_dia_instituto[aula['dia']]:
        agrupamento_dia_instituto[aula['dia']][aula['unidade']] = 0
    agrupamento_dia_instituto[aula['dia']][aula['unidade']] += aula['total_matriculados']
agrupamento_dia_instituto

{'seg': {'Instituto de Física': 97,
  'Instituto de Geociências': 51,
  'Escola Politécnica': 313,
  'Instituto de Matemática e Estatística': 183,
  'Instituto de Química': 81,
  'Instituto de Biociências': 94,
  'Faculdade de Odontologia': 82,
  'Escola de Educação Física e Esporte': 62,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 14,
  'Faculdade de Ciências Farmacêuticas': 26,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 110,
  'Faculdade de Medicina Veterinária e Zootecnia': 236,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 125},
 'ter': {'Instituto de Biociências': 91,
  'Escola Politécnica': 314,
  'Faculdade de Medicina Veterinária e Zootecnia': 168,
  'Instituto de Geociências': 102,
  'Instituto de Matemática e Estatística': 256,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 120,
  'Faculdade de Ciências Farmacêuticas': 18,
  'Faculdade de Filosofia, Letras e Ciênci

In [41]:
for dia in agrupamento_dia_instituto:
    for instituto in agrupamento_dia_instituto[dia]:
        if len(porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida']) > 0:
            agrupamento_dia_instituto[dia][instituto] *= porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_ida'].iloc[0]
        else:
            agrupamento_dia_instituto[dia][instituto] = 0
        agrupamento_dia_instituto[dia][instituto] = int(agrupamento_dia_instituto[dia][instituto])

# Pessoas por instituto que utilizam circular de acordo com a pesquisa
agrupamento_dia_instituto

{'seg': {'Instituto de Física': 97,
  'Instituto de Geociências': 51,
  'Escola Politécnica': 313,
  'Instituto de Matemática e Estatística': 183,
  'Instituto de Química': 81,
  'Instituto de Biociências': 94,
  'Faculdade de Odontologia': 82,
  'Escola de Educação Física e Esporte': 0,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 14,
  'Faculdade de Ciências Farmacêuticas': 26,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 0,
  'Faculdade de Medicina Veterinária e Zootecnia': 0,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 125},
 'ter': {'Instituto de Biociências': 91,
  'Escola Politécnica': 314,
  'Faculdade de Medicina Veterinária e Zootecnia': 0,
  'Instituto de Geociências': 102,
  'Instituto de Matemática e Estatística': 256,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 0,
  'Faculdade de Ciências Farmacêuticas': 18,
  'Faculdade de Filosofia, Letras e Ciências Humana